In [219]:
import copy
import datetime
import pandas as pd
import os
from tqdm import tqdm, trange
import pymssql
from optparse import OptionParser, Option, OptionValueError
import warnings
warnings.filterwarnings('ignore')

def get_data(database, command):
    
    db_setting = {'server' : '10.60.0.39', 
                  'user' : 'WarrantWeb',
                  'password' : 'WarrantWeb',
                  'database' : database,}
                  #'charset' : 'GBK'}

    conn = pymssql.connect(**db_setting)
    cursor = conn.cursor()
    data = pd.read_sql(command, conn)
    conn.close()

    return data

In [259]:
command = """ SELECT [TradeDate],[TraderId],[TraderNm],[strategymainnm],[strategysubnm],[IAccountId],[iaccountnm]
             ,[gaccountid],[CommodityId],[CommodityNm],[TradeType],[DataSource],[OldCommodityId],[CommodityKind],[CurrencyId]
             ,[ContractSize],[ExerciseSize],[CashDivAmt],[Unit],[IsTrade],[IssuerId],[PricingUnderlyingPrice],[BuyShare],[BuyAvgP]
             ,[BuyAmt],[SellShare],[SellAvgP],[SellAmt],[InvShare],[InvAvgP],[Cost],[MPrice],[MV],[UnRealPL],[DRealPL],[DUnRealPL]
             ,[DTax],[DSettleFee],[DCommission],[DStkBorrowFee],[DInterestCost],[DOtherIncome],[DCashDiv],[DTransferPL],[DTradePL]
             ,[DHTradePL],[DBTradePL],[MRealPL],[MUnRealPL],[MTax],[MSettleFee],[MCommission],[MStkBorrowFee],[MInterestCost],[MOtherIncome]
             ,[MCashDiv],[MTransferPL],[MTradePL],[MHTradePL],[MBTradePL],[YRealPL],[YUnRealPL],[YTax],[YSettleFee],[YCommission]
             ,[YStkBorrowFee],[YInterestCost],[YOtherIncome],[YCashDiv],[YTransferPL],[YTradePL],[YHTradePL],[YBTradePL],[AvailableShare]
             ,[CashDivAdj],[FxRate],[FxUnderlyingRate],[newIAccountId],[NewContractSize],[Forex],[Strategy]
             FROM [ETFData].[dbo].[vwDailyPL] where TradeDate = '2023-02-22' and gaccountid = 'LHD_TW' """
df = get_data('ETFData', command)
df['TraderId'] = df['TraderId'].str.encode('latin').str.decode('cp950')
df['TraderNm'] = df['TraderNm'].str.encode('latin').str.decode('cp950')
df['strategymainnm'] = df['strategymainnm'].str.encode('latin').str.decode('cp950')
df['strategysubnm'] = df['strategysubnm'].str.encode('latin').str.decode('cp950')
df['CommodityNm'] = df['CommodityNm'].str.encode('latin').str.decode('cp950')
df['size'] = df.apply(lambda x: x.ExerciseSize * -1 if '反1' in x.CommodityNm else x.ExerciseSize, axis = 1)
# df.to_csv('df.csv', index = False, encoding = 'big5')

In [260]:
command = """ SELECT [TradeDate],[TraderId],[TraderNm],[strategymainnm],[strategysubnm],[IAccountId],[iaccountnm]
             ,[gaccountid],[CommodityId],[CommodityNm],[TradeType],[DataSource],[OldCommodityId],[CommodityKind],[CurrencyId]
             ,[ContractSize],[ExerciseSize],[CashDivAmt],[Unit],[IsTrade],[IssuerId],[PricingUnderlyingPrice],[BuyShare],[BuyAvgP]
             ,[BuyAmt],[SellShare],[SellAvgP],[SellAmt],[InvShare],[InvAvgP],[Cost],[MPrice],[MV],[UnRealPL],[DRealPL],[DUnRealPL]
             ,[DTax],[DSettleFee],[DCommission],[DStkBorrowFee],[DInterestCost],[DOtherIncome],[DCashDiv],[DTransferPL],[DTradePL]
             ,[DHTradePL],[DBTradePL],[MRealPL],[MUnRealPL],[MTax],[MSettleFee],[MCommission],[MStkBorrowFee],[MInterestCost],[MOtherIncome]
             ,[MCashDiv],[MTransferPL],[MTradePL],[MHTradePL],[MBTradePL],[YRealPL],[YUnRealPL],[YTax],[YSettleFee],[YCommission]
             ,[YStkBorrowFee],[YInterestCost],[YOtherIncome],[YCashDiv],[YTransferPL],[YTradePL],[YHTradePL],[YBTradePL],[AvailableShare]
             ,[CashDivAdj],[FxRate],[FxUnderlyingRate],[newIAccountId],[NewContractSize],[Forex],[Strategy]
             FROM [ETFData].[dbo].[vwDailyPL] where TradeDate = '2023-02-23' and gaccountid = 'LHD_TW' """
                        
# command = """ SELECT *
#              FROM [ETFData].[dbo].[vwDailyPL] where TradeDate = '2023-02-22' and gaccountid = 'LHD_TW' """
df2 = get_data('ETFData', command)
df2['TraderId'] = df2['TraderId'].str.encode('latin').str.decode('cp950')
df2['TraderNm'] = df2['TraderNm'].str.encode('latin').str.decode('cp950')
df2['strategymainnm'] = df2['strategymainnm'].str.encode('latin').str.decode('cp950')
df2['strategysubnm'] = df2['strategysubnm'].str.encode('latin').str.decode('cp950')
df2['CommodityNm'] = df2['CommodityNm'].str.encode('latin').str.decode('cp950')
df2['size'] = df2.apply(lambda x: x.ExerciseSize * -1 if '反1' in x.CommodityNm else x.ExerciseSize, axis = 1)
# df2.to_csv('df2.csv', index = False, encoding = 'big5')


In [222]:
test1 = pd.merge(df2[['CommodityId', 'TradeType', 'Cost', 'MV', 'size', 'InvShare', 'InvAvgP', 'MPrice', 'BuyShare', 'BuyAvgP', 'SellShare', 'SellAvgP']], 
                 df[['CommodityId', 'TradeType', 'Cost', 'MV', 'InvShare', 'InvAvgP', 'MPrice', 'BuyShare', 'BuyAvgP', 'SellShare', 'SellAvgP', 'NewContractSize']],
                 how = 'left', on = ['CommodityId', 'TradeType'], suffixes = ['_t', '_0'])
test1

,CommodityId,TradeType,Cost_t,MV_t,size,InvShare_t,InvAvgP_t,MPrice_t,BuyShare_t,BuyAvgP_t,...,Cost_0,MV_0,InvShare_0,InvAvgP_0,MPrice_0,BuyShare_0,BuyAvgP_0,SellShare_0,SellAvgP_0,NewContractSize
0,0050,0,2419.0,2388.0,1.0,20,120.950000,119.40,0,0.000000,...,2419.0,2353.0,20,120.950000,117.65,0,0.000000,0,0.000000,1.0
1,00631L,0,1006700.0,1017900.0,2.0,9000,111.855500,113.10,0,0.000000,...,1006700.0,988200.0,9000,111.855500,109.80,0,0.000000,0,0.000000,1.0
2,00632R,0,0.0,0.0,-1.0,0,0.000000,5.33,0,0.000000,...,0.0,0.0,0,0.000000,5.41,0,0.000000,0,0.000000,1.0
3,00663L,0,17871450.0,18130400.0,2.0,173000,103.303100,104.80,0,0.000000,...,17871450.0,17559500.0,173000,103.303100,101.50,0,0.000000,0,0.000000,1.0
4,00664R,0,12584050.0,12479460.0,-1.0,2073000,6.070400,6.02,0,0.000000,...,13063580.0,13148720.0,2152000,6.070400,6.11,0,0.000000,0,0.000000,1.0
5,00675L,0,32117850.0,32709600.0,2.0,720000,44.608100,45.43,308000,45.271526,...,40652620.0,40441400.0,916000,44.380500,44.15,621000,43.861868,398000,43.945779,1.0
6,00685L,0,10070900.0,10117810.0,2.0,251000,40.123100,40.31,18000,40.178889,...,9347680.0,9119620.0,233000,40.118700,39.14,1000,38.760000,0,0.000000,1.0
7,00686R,0,0.0,0.0,-1.0,0,0.000000,3.59,0,0.000000,...,0.0,0.0,0,0.000000,3.64,0,0.000000,0,0.000000,1.0
8,0050,6,-80485870.0,-89550000.0,1.0,-750000,107.310000,119.40,0,0.000000,...,-80485870.0,-88237500.0,-750000,107.310000,117.65,0,0.000000,0,0.000000,1.0
9,00632R,6,-154115330.0,-152800440.0,-1.0,-28668000,5.370000,5.33,0,0.000000,...,-111628020.0,-112289960.0,-20756000,5.370000,5.41,0,0.000000,8166000,5.405108,1.0


### 總市值變化 (市值-昨日市值)

In [223]:
ETF = test1[0:17]
fut = test1[17:]
print('總市值變化 : ', (ETF['MV_t'] * ETF['size']).sum() + fut['MV_t'].sum() - ((ETF['MV_0'] * ETF['size']).sum() + fut['MV_0'].sum()))
print('今日市值 :', (ETF['MV_t'] * ETF['size']).sum() + fut['MV_t'].sum())
print('昨日市值 :', ((ETF['MV_0'] * ETF['size']).sum() + fut['MV_0'].sum()))

總市值變化 :  -296325.0
今日市值 : -121112.0
昨日市值 : 175213.0


In [224]:
(ETF['MV_t'] * ETF['size']).sum()

200451388.0

In [225]:
((ETF['MV_0'] * ETF['size'])).sum()

172000213.0

In [226]:
fut['MV_t'].sum()

-200572500.0

In [227]:
fut['MV_0'].sum()

-171825000.0

In [228]:
### 損益變化
    # 昨日庫存損益(今收-昨收) * 昨日庫存
    # 今日進出損益 (今買均 - 今收) * 今買量 + (今收 - 今賣均) * 今賣量


# ((ETF['MPrice_t'] - ETF['MPrice_0']) *  ETF['InvShare_0'] * ETF['size']).sum() + \
# ((ETF['BuyAvgP_t'] - ETF['MPrice_t']) * ETF['BuyShare_t'] * ETF['size']).sum() + \
# ((ETF['MPrice_t'] - ETF['SellAvgP_t']) * ETF['SellShare_t'] * ETF['size']).sum()
# ((ETF['MPrice_t'] - ETF['MPrice_0']) *  ETF['InvShare_0']).sum() + \
# ((ETF['BuyAvgP_t'] - ETF['MPrice_t']) * ETF['BuyShare_t']).sum() + \
# ((ETF['MPrice_t'] - ETF['SellAvgP_t']) * ETF['SellShare_t']).sum()

In [229]:
# ((fut['MPrice_t'] - fut['MPrice_0']) *  fut['InvShare_0'] * fut['NewContractSize']).sum() + \
# ((fut['BuyAvgP_t'] - fut['MPrice_t']) * fut['BuyShare_t'] * fut['NewContractSize']).sum() + \
# ((fut['MPrice_t'] - fut['SellAvgP_t']) * fut['SellShare_t'] * fut['NewContractSize']).sum()

In [230]:
# print('今日市值 :', (fut['MV_t'].sum()))
# print('昨日市值 :', (fut['MV_0'].sum()))

# print('今日市值 :', (ETF['MV_t'] * ETF['size']).sum())
# print('昨日市值 :', ((ETF['MV_0'] * ETF['size']).sum()))

### 計算Delta SPD

In [261]:
command = f"SELECT [DataDate],[ProductCode],[NAV],[Price] FROM [ETFData].[dbo].[vETFNAV] \
            where ProductCode in {tuple(test1['CommodityId'].unique())} and DataDate = '20230223' "
NAV_t = get_data('ETFData', command)

command = f"SELECT [DataDate],[ProductCode],[NAV],[Price] FROM [ETFData].[dbo].[vETFNAV] \
            where ProductCode in {tuple(test1['CommodityId'].unique())} and DataDate = '20230222' "
NAV_0 = get_data('ETFData', command)

# NAV_0.to_csv('NAV_0.csv', encoding = 'big5', index = False)
# NAV_t.to_csv('NAV_t.csv', encoding = 'big5', index = False)

In [236]:
test2 = pd.merge(test1, NAV_t[['ProductCode', 'NAV']], how = 'left', left_on = 'CommodityId', right_on = 'ProductCode').drop(columns = 'ProductCode')
test2 = test2.rename(columns = {'NAV':'NAV_t'})
column_to_move = test2.pop('NAV_t')
test2.insert(3, 'NAV_t', column_to_move )

test3 = pd.merge(test2, NAV_0[['ProductCode', 'NAV']], how = 'left', left_on = 'CommodityId', right_on = 'ProductCode').drop(columns = 'ProductCode')
test3 = test3.rename(columns = {'NAV':'NAV_0'})
column_to_move = test3.pop('NAV_0')
test3.insert(14, 'NAV_0', column_to_move)

In [238]:
ETF = test3[0:17]
ETF

,CommodityId,TradeType,Cost_t,NAV_t,MV_t,size,InvShare_t,InvAvgP_t,MPrice_t,BuyShare_t,...,NAV_0,MV_0,InvShare_0,InvAvgP_0,MPrice_0,BuyShare_0,BuyAvgP_0,SellShare_0,SellAvgP_0,NewContractSize
0,0050,0,2419.0,119.34,2388.0,1.0,20,120.9500,119.40,0,...,117.63,2353.0,20,120.9500,117.65,0,0.000000,0,0.000000,1.0
1,00631L,0,1006700.0,113.03,1017900.0,2.0,9000,111.8555,113.10,0,...,109.79,988200.0,9000,111.8555,109.80,0,0.000000,0,0.000000,1.0
2,00632R,0,0.0,5.33,0.0,-1.0,0,0.0000,5.33,0,...,5.40,0.0,0,0.0000,5.41,0,0.000000,0,0.000000,1.0
3,00663L,0,17871450.0,104.40,18130400.0,2.0,173000,103.3031,104.80,0,...,101.46,17559500.0,173000,103.3031,101.50,0,0.000000,0,0.000000,1.0
4,00664R,0,12584050.0,6.01,12479460.0,-1.0,2073000,6.0704,6.02,0,...,6.10,13148720.0,2152000,6.0704,6.11,0,0.000000,0,0.000000,1.0
5,00675L,0,32117850.0,45.41,32709600.0,2.0,720000,44.6081,45.43,308000,...,44.15,40441400.0,916000,44.3805,44.15,621000,43.861868,398000,43.945779,1.0
6,00685L,0,10070900.0,40.23,10117810.0,2.0,251000,40.1231,40.31,18000,...,39.11,9119620.0,233000,40.1187,39.14,1000,38.760000,0,0.000000,1.0
7,00686R,0,0.0,3.59,0.0,-1.0,0,0.0000,3.59,0,...,3.64,0.0,0,0.0000,3.64,0,0.000000,0,0.000000,1.0
8,0050,6,-80485870.0,119.34,-89550000.0,1.0,-750000,107.3100,119.40,0,...,117.63,-88237500.0,-750000,107.3100,117.65,0,0.000000,0,0.000000,1.0
9,00632R,6,-154115330.0,5.33,-152800440.0,-1.0,-28668000,5.3700,5.33,0,...,5.40,-112289960.0,-20756000,5.3700,5.41,0,0.000000,8166000,5.405108,1.0


### 損益變化
    昨日庫存損益(今收-昨收) * 昨日庫存
    今日進出損益 (今買均 - 今收) * 今買量 + (今收 - 今賣均) * 今賣量


In [250]:
#理論溢價
((ETF['MPrice_t'] - ETF['MPrice_0']) *  ETF['InvShare_0']).sum() + \
((ETF['BuyAvgP_t'] - ETF['MPrice_t']) * ETF['BuyShare_t']).sum() + \
((ETF['MPrice_t'] - ETF['SellAvgP_t']) * ETF['SellShare_t']).sum()

2257599.5039999993

In [249]:
((ETF['NAV_t'] - ETF['NAV_0']) *  ETF['InvShare_0']).sum() + \
((ETF['BuyAvgP_t'] - ETF['NAV_t']) * ETF['BuyShare_t']).sum() + \
((ETF['NAV_t'] - ETF['SellAvgP_t']) * ETF['SellShare_t']).sum()

1981798.703999998

In [255]:
print('庫存溢價 :',((ETF['MPrice_t'] - ETF['MPrice_0']) *  ETF['InvShare_0']).sum() - ((ETF['NAV_t'] - ETF['NAV_0']) *  ETF['InvShare_0']).sum())
print('今日買溢價 :',((ETF['BuyAvgP_t'] - ETF['MPrice_t']) * ETF['BuyShare_t']).sum() - ((ETF['BuyAvgP_t'] - ETF['NAV_t']) * ETF['BuyShare_t']).sum())
print('今日賣溢價 :',((ETF['BuyAvgP_t'] - ETF['NAV_t']) * ETF['BuyShare_t']).sum() - ((ETF['NAV_t'] - ETF['SellAvgP_t']) * ETF['SellShare_t']).sum())

庫存溢價 : 267570.80000000075
今日買溢價 : -7630.000000001062
今日賣溢價 : 136575.49600000578


In [265]:
command = """SELECT *
  FROM [ETFData].[dbo].[ETFTickData]
  where ETFID in ('00878', '00632R') and TDate = '20230222' """
zz = get_data('ETFData', command)

In [266]:
zz

,TDate,TTime,ETFID,NAVBid1,NAVAsk1,NAVTrade,MktBid1,MktAsk1,MktTrade,VolumeBid1,VolumeAsk1,VolumeTrade
0,2023-02-22,09:00:02.030000,00632R,5.4083,5.4095,5.4095,5.40,5.41,5.41,26914,2859,0
1,2023-02-22,09:00:02.030000,00878,16.9614,16.9978,16.9726,17.03,17.04,17.04,173,108,0
2,2023-02-22,09:00:07.043000,00632R,5.4087,5.4092,5.4089,5.40,5.41,5.41,41839,1292,9389
3,2023-02-22,09:00:07.043000,00878,16.9569,17.0002,16.9717,17.01,17.02,17.02,1,132,4943
4,2023-02-22,09:00:12.060000,00632R,5.4066,5.4076,5.4071,5.40,5.41,5.41,27439,1201,100
...,...,...,...,...,...,...,...,...,...,...,...,...
6487,2023-02-22,13:30:48.073000,00878,17.0262,17.0579,17.0383,17.05,17.06,17.06,2915,292,0
6488,2023-02-22,13:30:53.073000,00632R,5.4038,5.4045,5.4040,5.40,5.41,5.41,26658,1515,0
6489,2023-02-22,13:30:53.073000,00878,17.0262,17.0578,17.0383,17.05,17.06,17.06,2915,292,0
6490,2023-02-22,13:30:58.073000,00632R,5.4038,5.4045,5.4043,5.40,5.41,5.41,26658,1515,0


In [267]:
zz.to_csv('tick_.csv', index = False, encoding = 'ANSI')